# Gather iNaturalist data

I'm gathering up to 500 records of each of these class for every plant order:
1. Flowering only
1. Fruiting only
1. Both flowering and fruiting
1. Neither flowering nor fruiting. I may up this class to 1000 or 1500 if the models don't cooperate.

In [1]:
import json
import math
import re
import time
from pathlib import Path
from types import SimpleNamespace

import pandas as pd
import requests
from tqdm.notebook import tqdm

In [2]:
INAT_DIR = Path("..") / "data" / "inat"

In [3]:
args = SimpleNamespace(
    base_url="https://api.inaturalist.org/v1/observations?",
    taxa=INAT_DIR / "taxa.csv.gz",  # Download iNat taxon info here
    obs_dir=INAT_DIR / "obs",  # Save observation JSON data here
    image_dir=INAT_DIR / "images",  # Save images here
    plantae=47126,  # The taxon ID for the plant kingdom
    per_page=100,  # How many iNaturalist records to get per API request
    pages=5,  # Limit to this number of a pages
)

## Get target taxon data

### Download the iNaturalist taxon data using AWS command line.

In [4]:
# !aws s3 cp s3://inaturalist-open-data/taxa.csv.gz $args.taxa --no-sign-request

### Read in the taxon data I just downloaded to a data frame.

In [5]:
taxa_df = pd.read_csv(
    args.taxa,
    sep="\t",
    dtype={
        "taxon_id": int,
        "ancestry": str,
        "rank_level": float,
        "rank": str,
        "name": str,
        "active": bool,
    },
).fillna("")
taxa_df.shape

(1394173, 6)

In [6]:
taxa_df = taxa_df.loc[taxa_df["ancestry"].str.contains(f"/{args.plantae}/")]
taxa_df.head()

,taxon_id,ancestry,rank_level,rank,name,active
18999,47352,48460/47126/211194/47125/47124/47132/47148/922...,10.0,hybrid,Prunus × yedoensis,True
19008,49389,48460/47126/211194/47125/47124/47366/48567/791419,20.0,genus,Stellaria,True
19042,49310,48460/47126/211194/47125/47124/47181/48064/791...,5.0,subspecies,Dodecatheon hendersonii cruciatum,False
19061,49077,48460/47126/211194/47125/47124/48808/49118/491...,5.0,variety,Chimonanthus praecox intermedius,False
19076,48101,48460/47126/211194/47125/47124/47729/58321/477...,5.0,form,Acer rubrum pallidiflorum,False


In [7]:
orders = taxa_df.loc[taxa_df["rank"] == "order"]
orders = orders.set_index("name")["taxon_id"].to_dict()
len(orders)

261

## Make API requests for observation data to iNaturalist

### The URL for the API request to iNaturalist.

In [8]:
VALUES = [13, 14, 21]


def get_url(taxon_id, values, page=1, per_page=args.per_page):
    withs = [str(v) for v in VALUES if v in values]
    withouts = [str(v) for v in VALUES if v not in values]

    parts = [
        args.base_url,
        "photos=true",
        "quality_grade=research",
        f"taxon_id={taxon_id}",
        f"per_page={per_page}",
        f"page={page}",
        "order=desc",
        "order_by=created_at",
        "photo_license=cc0,cc-by,cc-by-nc,cc-by-sa",
        "term_id=12",
        f"term_value_id={','.join(withs)}",
        f"without_term_value_id={','.join(withouts)}",
    ]
    url = "&".join(parts)
    return url

### Make the API request.

In [9]:
def call_api(url, results):
    try:
        response = requests.get(url)
        results += response.json()["results"]
    except KeyError:
        print(response.json())
        raise  # ???????????????????????????
    return response.json()

### Loop through each plant order and make requests to the API

Right now I'm capping the total number of records per plant order/phenology class.

In [13]:
terms = [
    {"term": "flowering", "values": [13]},
    {"term": "fruiting", "values": [14]},
    {"term": "neither", "values": [21]},
    {"term": "both", "values": [13, 14]},
]

for order, taxon_id in tqdm(orders.items(), position=0, leave=None):

    for term in terms:

        path = args.obs_dir / f"{term['term']}_{order}.json"
        if path.exists:
            continue

        url = get_url(taxon_id, term["values"])

        results = []
        response = call_api(url, results)

        if len(results) == 0:
            continue

        last = math.ceil(response["total_results"] / args.per_page) + 1
        last = min(last, args.pages + 1)

        for page in tqdm(range(2, last), position=1, leave=None, desc=term["term"]):
            url = get_url(taxon_id, term["values"], page=page)
            call_api(url, results)

        with open(path, "w") as out_file:
            json.dump(results, out_file)

  0%|          | 0/261 [00:00<?, ?it/s]

## Download images

### Try the image download a few times.

In [14]:
def download_image(url, path):
    if path.exists():
        return
    for attempt in range(3):
        try:
            image = requests.get(url).content
            with open(path, "wb") as out_file:
                out_file.write(image)
            return
        except (TimeoutError, ConnectionError):
            time.sleep(20)
    else:
        raise TimeoutError

### Loop through each set of downloaded JSON observations and get their images.

In [19]:
size = "medium."  # medium large original

json_paths = sorted(args.obs_dir.glob("*.json"))

for json_path in tqdm(json_paths, position=0, leave=None):

    with open(json_path) as in_file:
        data = json.load(in_file)

    for result in tqdm(data, position=1, leave=None, desc=json_path.stem):

        for photo in result["photos"]:
            url = photo["url"].replace("square.", size)

            match = re.search(r"/(\d+)/[a-z]+\.([a-z]+)$", url, flags=re.I)
            if not match:
                continue

            image_path = args.image_dir / f"{match[1]}_{size}{match[2]}"
            download_image(url, image_path)

  0%|          | 0/370 [00:00<?, ?it/s]

both_Acorales:   0%|          | 0/56 [00:00<?, ?it/s]

both_Alismatales:   0%|          | 0/500 [00:00<?, ?it/s]

both_Apiales:   0%|          | 0/500 [00:00<?, ?it/s]

both_Aquifoliales:   0%|          | 0/500 [00:00<?, ?it/s]

...

pheno_neither_order_Santalales:   0%|          | 0/500 [00:00<?, ?it/s]

pheno_neither_order_Saxifragales:   0%|          | 0/500 [00:00<?, ?it/s]

pheno_neither_order_Solanales:   0%|          | 0/500 [00:00<?, ?it/s]

pheno_neither_order_Zingiberales:   0%|          | 0/281 [00:00<?, ?it/s]

pheno_neither_order_Zygophyllales:   0%|          | 0/500 [00:00<?, ?it/s]